# Product as summation

A product of values can be treated as a summation of the logs of the values due to the following log identity:
* $log(ab) = log(a)+log(b)

This can useful for SQL aggregations where a product operation not supported.

Algorithm
1. apply a log to all values
    * values must be > 0, this is violated by logodds which centers scores around 0.5
        * add tiny value if the value is supposed to be zero
        * make value positive if it is negative
1. add all the values together
1. undo the log transform using and exponentiation
1. apply flip the sign if necessary to undo the sign flipping we did prior to the log tranform 



In [2]:
import math
import numpy as np

In [1]:
values = [.5, .1, .8]
values2 = [.1, .1, .1]
values3 = [.9, .9, .9]
oddsValues = [.1, .1, 900]
logOddsValues = [.0, .0, -900]
logOddsValues2 = [1.0, 2.0, -50]
logOddsValues3 = [1.0, -2.0, -50]

In [55]:
def productAgg(valueList):
    result = reduce(lambda a, b: a*b, valueList)
    return result

def logSumAgg(valueList):
    
    # count negative numbers
    numNegatives = sum(x < 0 for x in valueList)
    
    # need to handle zeros and negative numbers
    # so that the log transform doesn't blow up
    def handleEqLtZero(value):
        TOL = 0.000000000001
        result = value
        
        # add a tiny value if we are at 0
        if result == 0:
            result += TOL
        #  add make the value positive if it is negative
        elif result < 0:
            result = abs(result)
        
        return result
    
    logs = map(lambda e: math.log(handleEqLtZero(e)), valueList)
    sums = sum(logs)
    prod = math.exp(sums)
    
    # flip sign if necessary (when there was an odd number of negative values)
    if (numNegatives % 2) != 0:
        prod *= -1
    
    return prod

In [48]:
print(productAgg(values), logSumAgg(values))

(0.04000000000000001, 0.04000000000000001)


In [49]:
print(productAgg(values2), logSumAgg(values2))

(0.0010000000000000002, 0.0010000000000000002)


In [50]:
print(productAgg(oddsValues), logSumAgg(oddsValues))

(9.000000000000002, 9.000000000000005)


# Negative numbers

In [51]:
# Need to flip the sign if there is an odd number of negative numbers to be multiplied
print(productAgg(logOddsValues2), logSumAgg(logOddsValues2))

(-100.0, -99.99999999999996)


In [52]:
print(productAgg(logOddsValues3), logSumAgg(logOddsValues3))

(100.0, 99.99999999999996)


# zero values

In [53]:
print(productAgg(logOddsValues), logSumAgg(logOddsValues))

(-0.0, -8.999999999999994e-22)
